In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, Lasso
from scipy.stats import randint, uniform

In [2]:
#Step 2: Load Dataset
df = pd.read_csv("../data/Kangaroo.csv")

### Cleaning data

- EPC Score: remove the irrelevant symbol and mapping of EPC score (A -> G) into the mean of EPC classification (kWh/m2.year) by region (Wallonia / Flanders / Brussels). 
        df[epcScore] (object)  -> df[epc_enum] (float64)
- Drops some columns missing 100% data and unneccessary 
- Drop missing values of these important features: 'price','habitableSurface','bedroomCount','bathroomCount','epcScore'
- 
Removing Aberrant Values (Outlier Removal): based on the max of

In [3]:
# mapping epcScore for each province
def map_epc_score(df):
    epc_invalid = ['C_A', 'F_C', 'G_C', 'D_C', 'F_D', 'E_C', 'G_E', 'E_D', 'C_B', 'X', 'G_F']
    df = df[~df['epcScore'].isin(epc_invalid)].copy()

    wallonia_provinces = ['Liège', 'Walloon Brabant', 'Namur', 'Hainaut', 'Luxembourg']
    flanders_provinces = ['Antwerp', 'Flemish Brabant', 'East Flanders', 'West Flanders', 'Limburg']

    epc_maps = {
        "Wallonia": {'A++': 0, 'A+': 30, 'A': 65, 'B': 125, 'C': 200, 'D': 300, 'E': 375, 'F': 450, 'G': 510},
        "Flanders": {'A++': 0, 'A+': 0, 'A': 50, 'B': 150, 'C': 250, 'D': 350, 'E': 450, 'F': 500, 'G': 510},
        "Brussels": {'A++': 0, 'A+': 0, 'A': 45, 'B': 75, 'C': 125, 'D': 175, 'E': 250, 'F': 300, 'G': 350}
    }

    def map_score(row):
        if row['province'] in wallonia_provinces:
            return epc_maps['Wallonia'].get(row['epcScore'], None)
        elif row['province'] in flanders_provinces:
            return epc_maps['Flanders'].get(row['epcScore'], None)
        elif row['province'] == 'Brussels':
            return epc_maps['Brussels'].get(row['epcScore'], None)
        return None

    df.loc[:, 'epc_enum'] = df.apply(map_score, axis=1)
    return df

# Cleaning Function
def cleaning(df):
    # Drop columns missing 100 % values and unnecessary columns
    df = df.drop(columns=["Unnamed: 0", "url", "id", "monthlyCost", "accessibleDisabledPeople", "hasBalcony"])
    
    # Drop columns not important
    drop_cols = [
        'roomCount', 'diningRoomSurface', 'streetFacadeWidth', 'kitchenSurface', 'hasBasement', 'hasArmoredDoor',
        'floorCount', 'hasDiningRoom', 'hasDressingRoom', 'gardenSurface', 'terraceSurface', 'livingRoomSurface',
        'gardenOrientation', 'heatingType', 'kitchenType', 'terraceOrientation'
    ]
    df = df.drop(columns=drop_cols)
    
    # Drop rows with missing target or essential features
    df = df.dropna(axis=0, subset=['price','habitableSurface','bedroomCount','bathroomCount','epcScore'])

    # Convert boolean columns to binary (0/1)
    binary_cols = [
        'hasLift', 'hasHeatPump', 'hasPhotovoltaicPanels', 'hasAirConditioning', 'hasVisiophone', 'hasOffice',
        'hasSwimmingPool', 'hasFireplace', 'hasAttic', 'parkingCountIndoor', 'parkingCountOutdoor'
    ]
    for col in binary_cols:
        df[col] = df[col].map({True: 1, False: 0, 'True': 1, 'False': 0}).fillna(0).astype(int)


    # For facadeCount, rename and fill missing values based on subtype and remove rows with more than 4 facades
    df = df[df['facedeCount'] <= 4]  # Remove rows with more than 4 facades
    df['facadeCount'] = df['facedeCount']
    df = df.drop(columns='facedeCount')

    apartment_subtypes = ['APARTMENT', 'FLAT_STUDIO', 'GROUND_FLOOR', 'PENTHOUSE', 'APARTMENT_BLOCK']
    df.loc[df['subtype'].isin(apartment_subtypes), 'facadeCount'] = df.loc[df['subtype'].isin(apartment_subtypes), 'facadeCount'].fillna(1)

    house_subtypes = ['HOUSE', 'VILLA', 'DUPLEX', 'TOWN_HOUSE', 'MANSION']
    df.loc[df['subtype'].isin(house_subtypes), 'facadeCount'] = df.loc[df['subtype'].isin(house_subtypes), 'facadeCount'].fillna(3)

    larger_house_subtypes = ['EXCEPTIONAL_PROPERTY', 'BUNGALOW', 'COUNTRY_COTTAGE', 'TRIPLEX', 'CHALET', 'CASTLE', 'MANOR_HOUSE']
    df.loc[df['subtype'].isin(larger_house_subtypes), 'facadeCount'] = df.loc[df['subtype'].isin(larger_house_subtypes), 'facadeCount'].fillna(4)

    other_subtypes = ['MIXED_USE_BUILDING', 'SERVICE_FLAT', 'KOT', 'FARMHOUSE', 'LOFT', 'OTHER_PROPERTY']
    df.loc[df['subtype'].isin(other_subtypes), 'facadeCount'] = df.loc[df['subtype'].isin(other_subtypes), 'facadeCount'].fillna(2)

    condition_mapping = {
        'AS_NEW': 0, 'JUST_RENOVATED': 1, 'GOOD': 2,
        'TO_RENOVATE': 3, 'TO_RESTORE': 4, 'TO_BE_DONE_UP': 5
    }
    df['buildingCondition_mapping'] = df['buildingCondition'].map(condition_mapping)

    flood_mapping = {
        "NON_FLOOD_ZONE": 0,
        "POSSIBLE_N_CIRCUMSCRIBED_WATERSIDE_ZONE": 1,
        "CIRCUMSCRIBED_WATERSIDE_ZONE": 2,
        "POSSIBLE_N_CIRCUMSCRIBED_FLOOD_ZONE": 3,
        "POSSIBLE_FLOOD_ZONE": 4,
        "CIRCUMSCRIBED_FLOOD_ZONE": 5,
        "RECOGNIZED_FLOOD_ZONE": 6,
        "RECOGNIZED_N_CIRCUMSCRIBED_WATERSIDE_FLOOD_ZONE": 7,
        "RECOGNIZED_N_CIRCUMSCRIBED_FLOOD_ZONE": 8
    }
    df['floodZoneType_mapping'] = df['floodZoneType'].map(flood_mapping)

    return df

# # Add postcode mean price feature
# def add_postcode_price_mean(df):
#     postcode_mean = df.groupby('postCode')['price'].mean().to_dict()
#     df['postcode_price_mean'] = df['postCode'].map(postcode_mean)
#     return df

# ======= ADD price_per_m2 and postcode average price per m2 =======
def add_postcode_price_mean(df):
    # Calculate price per m2
    df['price_per_m2'] = df['price'] / df['habitableSurface']
    
    # Compute average price_per_m2 per postcode
    postcode_price = df.groupby('postCode')['price_per_m2'].mean().reset_index()
    postcode_price.rename(columns={'price_per_m2': 'postcode_avg_price_per_m2'}, inplace=True)
    
    # Merge the average back to original df
    df = df.merge(postcode_price, on='postCode', how='left')
    
    return df


In [4]:
# Apply all preprocessing
df = map_epc_score(df)
df = cleaning(df)
df = add_postcode_price_mean(df)

In [5]:
# --- Feature limits for manual outlier removal ---
feature_limits = {
    'price': (100000, 800000),
    'habitableSurface': (20, 500),
    'bathroomCount': (1, 10),
    'bedroomCount': (1, 10),
    #'landSurface': (0, 10000),  # Only for houses
}

def apply_feature_limits(df, limits):
    df_filtered = df.copy()
    for feature, (min_val, max_val) in limits.items():
        df_filtered = df_filtered[(df_filtered[feature] >= min_val) & (df_filtered[feature] <= max_val)]
    return df_filtered

# --- Apply limits to remove outliers ---
df = apply_feature_limits(df, feature_limits)

In [8]:
# --- Data Preprocessing ---
# Ensure boolean features are binary
boolean_features = [
    'hasLift', 'hasHeatPump', 'hasPhotovoltaicPanels', 'hasAirConditioning', 
    'hasVisiophone', 'hasOffice', 'hasSwimmingPool', 'hasFireplace', 'hasAttic', 
    'parkingCountIndoor', 'parkingCountOutdoor'
]
df[boolean_features] = df[boolean_features].astype(int)

# Define feature groups
numerical_features = [
    'habitableSurface', 'bedroomCount', 'bathroomCount', 'facadeCount',
    'landSurface', 'buildingConstructionYear', 'epc_enum', 'floodZoneType_mapping', 'buildingCondition_mapping', 
    'postcode_avg_price_per_m2'
]
categorical_features = [
    'type', 'subtype', 'province'
]
# Define all features and target
all_features = numerical_features + categorical_features + boolean_features
X = df[all_features]
y = df['price']


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing pipelines
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, numerical_features),
    ('cat', categorical_pipeline, categorical_features),
    ('bool', 'passthrough', boolean_features)
])


In [ ]:

# Models to compare
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42),
    'Neural Network (MLP)': MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
}

# Evaluate each model
for model_name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f"{model_name}:")
    print(f"  MAE  = {mae:.2f}")
    print(f"  RMSE = {rmse:.2f}")
    print(f"  R²   = {r2:.4f}")
    print("-" * 40)

    

Linear Regression:
  MAE  = 62644.74
  RMSE = 84893.49
  R²   = 0.7094
----------------------------------------
Random Forest:
  MAE  = 49413.71
  RMSE = 69878.03
  R²   = 0.8031
----------------------------------------
Gradient Boosting:
  MAE  = 53759.18
  RMSE = 73624.76
  R²   = 0.7814
----------------------------------------
XGBoost:
  MAE  = 49603.09
  RMSE = 69209.85
  R²   = 0.8068
----------------------------------------
Neural Network (MLP):
  MAE  = 61490.90
  RMSE = 83569.54
  R²   = 0.7184
----------------------------------------


c:\Users\Dieu\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


### summary results of some tests for preprocessing with the accuracy of different models
1. # for the same features: 
all_features = numerical_features + categorical_features + boolean_features
boolean_features = [
    'hasLift', 'hasHeatPump', 'hasPhotovoltaicPanels', 'hasAirConditioning', 
    'hasVisiophone', 'hasOffice', 'hasSwimmingPool', 'hasFireplace', 'hasAttic', 
    'parkingCountIndoor', 'parkingCountOutdoor'
]
df[boolean_features] = df[boolean_features].astype(int)

# Define feature groups
numerical_features = [
    'habitableSurface', 'bedroomCount', 'bathroomCount', 'facadeCount',
    'landSurface', 'buildingConstructionYear', 'epc_enum', 'floodZoneType_mapping', 'buildingCondition_mapping', 
    'postcode_avg_price_per_m2'
]
categorical_features = [
    'type', 'subtype', 'province'
]


# 1.1. using median and most frequent for num (scaler, StandardScaler) and cat features (encoder, OneHotEncoder(handle_unknown='ignore') )
### Result: 
Linear Regression - Mean Absolute Error: 181352.09
Random Forest - Mean Absolute Error: 90434.27
Gradient Boosting - Mean Absolute Error: 104593.16
XGBoost - Mean Absolute Error: 94510.87
Neural Network (MLP) - Mean Absolute Error: 183473.89

# 1.2. Add outlier remove function: def apply_feature_limits(df, limits) 
feature_limits = {
    'price': (100000, 1000000),
    'habitableSurface': (20, 500),
    'bathroomCount': (1, 10),
    'bedroomCount': (1, 10),
    'landSurface': (10, 10000),

Linear Regression - Mean Absolute Error: 67607.64
Random Forest - Mean Absolute Error: 59041.74
Gradient Boosting - Mean Absolute Error: 55645.81
XGBoost - Mean Absolute Error: 53134.02
Neural Network (MLP) - Mean Absolute Error: 69159.10

# add apartment
 feature_limits = {
    'price': (100000, 800000),
    'habitableSurface': (20, 500),
    'bathroomCount': (1, 10),
    'bedroomCount': (1, 10),

Linear Regression - Mean Absolute Error: 62644.74
Random Forest - Mean Absolute Error: 49413.71
Gradient Boosting - Mean Absolute Error:  53759.18
XGBoost - Mean Absolute Error:  49603.09
Neural Network (MLP) - Mean Absolute Error: 61490.90

# the range of feature_limits is small, the better accuracy 
feature_limits = {
    'price': (100000, 800000),
    'habitableSurface': (20, 800),
    'bathroomCount': (1, 10),
    'bedroomCount': (1, 10),
    'landSurface': (10, 10000),
}
Linear Regression - Mean Absolute Error: 61062.43
Random Forest - Mean Absolute Error: 54766.16
Gradient Boosting - Mean Absolute Error: 56571.47
XGBoost - Mean Absolute Error: 54081.98
Neural Network (MLP) - Mean Absolute Error: 69936.27


feature_limits = {
    'price': (100000, 800000),
    'habitableSurface': (20, 500),
    'bathroomCount': (1, 10),
    'bedroomCount': (1, 10),
    'landSurface': (10, 10000),
}
Linear Regression - Mean Absolute Error: 60233.06
Random Forest - Mean Absolute Error: 53438.65
Gradient Boosting - Mean Absolute Error: 52555.35
XGBoost - Mean Absolute Error: 54081.98
Neural Network (MLP) - Mean Absolute Error: 68686.37



### Improve the accuracy 
1. Train different models on subsets (for ex., separate models for houses vs apartments) and Use ensemble methods like stacking --> little bit better 


In [228]:
boolean_features = [
    'hasLift', 'hasHeatPump', 'hasPhotovoltaicPanels', 'hasAirConditioning', 
    'hasVisiophone', 'hasOffice', 'hasSwimmingPool', 'hasFireplace', 'hasAttic', 
    'parkingCountIndoor', 'parkingCountOutdoor'
]

numerical_features = [
    'habitableSurface', 'bedroomCount', 'bathroomCount', 'facadeCount',
    'landSurface', 'buildingConstructionYear', 'epc_enum', 'floodZoneType_mapping', 'buildingCondition_mapping', 
    'postcode_avg_price_per_m2'
]

categorical_features = ['type', 'subtype', 'province']

all_features = numerical_features + categorical_features + boolean_features

models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, eval_metric='rmse')
}

def train_and_evaluate(df_subset, subset_name):
    if df_subset.empty:
        print(f"No data available for {subset_name}. Skipping...")
        return
    
    X = df_subset[all_features]
    y = df_subset['price']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    print(f"\nModel results for {subset_name}:")
    for model_name, model in models.items():
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('model', model)
        ])
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)

        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        print(f"{subset_name:<12} | {model_name:<18} | MAE = {mae:10.2f} | RMSE = {rmse:10.2f} | R² = {r2:6.4f}")

# --- Stacking model ---
def stacking_example(df_subset, subset_name):
    if df_subset.empty:
        print(f"No data available for {subset_name}. Skipping stacking...")
        return

    X = df_subset[all_features]
    y = df_subset['price']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    estimators = [
        ('rf', RandomForestRegressor(random_state=42)),
        ('gb', GradientBoostingRegressor(random_state=42)),
        ('xgb', XGBRegressor(random_state=42, eval_metric='rmse'))
    ]

    stacking_model = StackingRegressor(
        estimators=estimators,
        final_estimator=LinearRegression()
    )

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('stacking', stacking_model)
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f"{subset_name:<12} | Stacking Model      | MAE = {mae:10.2f} | RMSE = {rmse:10.2f} | R² = {r2:6.4f}")

# --- Apply to subsets ---
df_houses = df[df['type'].str.strip().str.lower() == 'house']
df_apartments = df[df['type'].str.strip().str.lower() == 'apartment']

train_and_evaluate(df_houses, "Houses")
train_and_evaluate(df_apartments, "Apartments")

stacking_example(df_houses, "Houses")
stacking_example(df_apartments, "Apartments")


Model results for Houses:
Houses       | Linear Regression  | MAE =   63367.09 | RMSE =   86409.43 | R² = 0.7083
Houses       | Random Forest      | MAE =   55108.82 | RMSE =   76377.77 | R² = 0.7721
Houses       | Gradient Boosting  | MAE =   58088.23 | RMSE =   78423.54 | R² = 0.7597
Houses       | XGBoost            | MAE =   54925.60 | RMSE =   75176.35 | R² = 0.7792

Model results for Apartments:
Apartments   | Linear Regression  | MAE =   52682.06 | RMSE =   74412.28 | R² = 0.7129
Apartments   | Random Forest      | MAE =   42083.38 | RMSE =   62258.99 | R² = 0.7990
Apartments   | Gradient Boosting  | MAE =   45298.70 | RMSE =   63918.68 | R² = 0.7881
Apartments   | XGBoost            | MAE =   42718.52 | RMSE =   62000.11 | R² = 0.8007


KeyboardInterrupt: 

In [ ]:
# Updated models with enhanced hyperparameters for RF and XGBoost
models = {
    'Ridge Regression': Ridge(alpha=1.0, random_state=42), 
    'Lasso Regression': Lasso(alpha=0.1, random_state=42, max_iter=10000), 
    'Random Forest': RandomForestRegressor(
        n_estimators=200,        # more trees
        max_depth=None,            # control tree depth
        min_samples_split=2,     # min samples to split
        min_samples_leaf=2,      # min samples per leaf
        max_features='auto',     # number of features to consider at each split
        random_state=42
    ),
    'XGBoost': XGBRegressor(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.05,      # smaller learning rate
        subsample=0.8,           # row sampling
        colsample_bytree=0.8,    # feature sampling
        random_state=42,
        eval_metric='rmse'
    )

}

for model_name, model in models.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"{model_name:<12} |     | MAE = {mae:10.2f} | RMSE = {rmse:10.2f} | R² = {r2:6.4f}")

Ridge Regression |     | MAE =   62609.71 | RMSE =   84868.62 | R² = 0.7095


c:\Users\Dieu\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.333e+13, tolerance: 6.555e+10
  model = cd_fast.enet_coordinate_descent(
c:\Users\Dieu\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Lasso Regression |     | MAE =   62614.18 | RMSE =   84874.11 | R² = 0.7095
Random Forest |     | MAE =   49538.75 | RMSE =   69990.77 | R² = 0.8024
XGBoost      |     | MAE =   47941.70 | RMSE =   67010.05 | R² = 0.8189


## result for Random Forest |     | MAE =   52373.80 | RMSE =   72401.40 | R² = 0.7886
Random Forest': RandomForestRegressor(
        n_estimators=200,        # more trees
        max_depth=20,            # control tree depth
        min_samples_split=5,     # min samples to split
        min_samples_leaf=2,      # min samples per leaf
        max_features='sqrt',     # number of features to consider at each split
        random_state=42
    ),
Lasso Regression |     | MAE =   62614.18 | RMSE =   84874.11 | R² = 0.7095

In [256]:
# Parameter distributions for Random Forest
rf_param_dist = {
    'model__n_estimators': randint(100, 500),
    'model__max_depth': randint(5, 30),
    'model__min_samples_split': randint(2, 10),
    'model__min_samples_leaf': randint(1, 10),
    'model__max_features': ['auto', 'sqrt', 'log2']
}

# Parameter distributions for XGBoost
xgb_param_dist = {
    'model__n_estimators': randint(100, 500),
    'model__max_depth': randint(3, 15),
    'model__learning_rate': uniform(0.01, 0.3),
    'model__subsample': uniform(0.6, 0.4),
    'model__colsample_bytree': uniform(0.6, 0.4),
    'model__gamma': uniform(0, 5)
}

# Create pipelines
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

xgb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(random_state=42, eval_metric='rmse'))
])

# RandomizedSearchCV for Random Forest
rf_search = RandomizedSearchCV(
    rf_pipeline,
    param_distributions=rf_param_dist,
    n_iter=50,
    cv=3,
    scoring='neg_mean_absolute_error',
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# RandomizedSearchCV for XGBoost
xgb_search = RandomizedSearchCV(
    xgb_pipeline,
    param_distributions=xgb_param_dist,
    n_iter=50,
    cv=3,
    scoring='neg_mean_absolute_error',
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit Random Forest
print("Tuning Random Forest...")
rf_search.fit(X_train, y_train)

print("Best Random Forest params:", rf_search.best_params_)
print(f"Best RF CV MAE: {-rf_search.best_score_:.2f}")

# Fit XGBoost
print("Tuning XGBoost...")
xgb_search.fit(X_train, y_train)

print("Best XGBoost params:", xgb_search.best_params_)
print(f"Best XGB CV MAE: {-xgb_search.best_score_:.2f}")

# Evaluate best models on test set
best_rf_model = rf_search.best_estimator_
best_xgb_model = xgb_search.best_estimator_

for model_name, model in [('Random Forest', best_rf_model), ('XGBoost', best_xgb_model)]:
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"{model_name} | MAE = {mae:.2f} | RMSE = {rmse:.2f} | R² = {r2:.4f}")

Tuning Random Forest...
Fitting 3 folds for each of 50 candidates, totalling 150 fits


c:\Users\Dieu\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Best Random Forest params: {'model__max_depth': 28, 'model__max_features': 'auto', 'model__min_samples_leaf': 1, 'model__min_samples_split': 3, 'model__n_estimators': 229}
Best RF CV MAE: 51758.25
Tuning XGBoost...
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best XGBoost params: {'model__colsample_bytree': 0.7710164073434198, 'model__gamma': 0.12709563372047594, 'model__learning_rate': 0.042367428097991336, 'model__max_depth': 9, 'model__n_estimators': 340, 'model__subsample': 0.7257423924305306}
Best XGB CV MAE: 48914.40
Random Forest | MAE = 49300.83 | RMSE = 69685.05 | R² = 0.8042
XGBoost | MAE = 46852.28 | RMSE = 65754.21 | R² = 0.8256
